In [1]:
import numpy as np

In [2]:
n_samples = 1000

In [3]:
mean = [5, 9]
cov = [[2.5, 0.8], [0.8, 0.5]]
X_p = np.random.multivariate_normal(mean, cov, n_samples).T
X_p.shape

(2, 1000)

In [4]:
mean = [11, 3]
cov = [[3, -1.3], [-1.3, 1.2]]
X_n_1 = np.random.multivariate_normal([11, 3], cov, int(n_samples/2)).T
X_n_2 = np.random.multivariate_normal([5, 2], cov, n_samples-int(n_samples/2)).T
X_n = np.hstack([X_n_1, X_n_2])
X_n.shape

(2, 1000)

In [5]:
XY_p = np.vstack([X_p, np.ones_like(X_p[0])])
XY_n = np.vstack([X_n, np.zeros_like(X_n[0])])
XY_n.shape

(3, 1000)

In [6]:
XY = np.hstack([XY_n, XY_p])
data_XY = np.copy(XY).T
np.random.shuffle(data_XY)
data_train = data_XY[:1600]
data_test = data_XY[:400]
data_train.shape

(1600, 3)

In [7]:
#params_initial = np.array([[-3], [3]])
params_initial = np.array([[1], [1]])
params_initial

array([[1],
       [1]])

In [8]:
class Logistic_regression_grad():
    
    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def _compute_loss(self, y, h, epsilon: float = 1e-5):
         
        # calculate binary cross entropy as loss 
        loss = (1/self.batch_size)*(((-y).T @ np.log(h + epsilon))-((1-y).T @ np.log(1-h + epsilon)))
        
        return loss
    
    def train(self, x, y, epochs, batch_size, lr):
        row, col = x.shape
        
        # Initializing weights and bias
        self.w = np.zeros((col,1))
        self.w0 = 1
        self.batch_size = batch_size
        self.lr = lr
        
        # defining bath size
        num_batches = x.shape[0]//self.batch_size
        
        
        for epoch in range(epochs):
            print("epoch: ", epoch)
            for batch_num in range(num_batches+1):
                
                # slicing batch data
                start = batch_num * self.batch_size
                end = (batch_num + 1) * self.batch_size
                
                x_batched = x[start:end]
                y_batched = np.array(y[start:end]).reshape((-1, 1))
                
                # predict for the epoch and batch
                # at first iteration we are using initial w/theta
                y_hat = self._sigmoid(np.dot(x_batched, self.w) + self.w0)
                
                # calculate gradient for weigths/theta
                error = y_hat - y_batched
        
                gradient_w = (1/self.batch_size)*np.dot(x_batched.T, error)
                gradient_w0 = (1/self.batch_size)*np.sum(error) 
                
                # adjusting weights/theta with learning rate annd calculated gradient 
                self.w -= self.lr*gradient_w
                self.w0 -= self.lr*gradient_w0
                
            # loss compute per epoch
            loss = self._compute_loss(y, self._sigmoid(np.dot(x, self.w) + self.w0))
            print("loss: ",loss)
        
        return self.w, self.w0        
    
    def predict(self, x_test):
        
        # predict on text data with calculated weigths/theta
        predictions = self._sigmoid(np.dot(x_test, self.w) + self.w0)
        
        # rounding up values to get classes
        predictions = np.round(predictions)
        
        return predictions

In [9]:
x_train = data_train[:,0:2]
y_train = data_train[:,2]

In [10]:
model = Logistic_regression_grad()

In [11]:
model.train(x_train,y_train,20,100,0.01)

epoch:  0
loss:  [7.29865126]
epoch:  1
loss:  [5.5776989]
epoch:  2
loss:  [4.71298937]
epoch:  3
loss:  [4.19097032]
epoch:  4
loss:  [3.83881459]
epoch:  5
loss:  [3.58330204]
epoch:  6
loss:  [3.38821848]
epoch:  7
loss:  [3.23358555]
epoch:  8
loss:  [3.10745648]
epoch:  9
loss:  [3.00223211]
epoch:  10
loss:  [2.91283711]
epoch:  11
loss:  [2.8357454]
epoch:  12
loss:  [2.7684262]
epoch:  13
loss:  [2.70901281]
epoch:  14
loss:  [2.65609602]
epoch:  15
loss:  [2.60859054]
epoch:  16
loss:  [2.56564589]
epoch:  17
loss:  [2.52658539]
epoch:  18
loss:  [2.49086332]
epoch:  19
loss:  [2.4580343]


(array([[-0.73759714],
        [ 0.66128902]]),
 0.8239512026802385)

In [12]:
data_test = data_XY[:400]
x_test = data_test[:,0:2]
y_test = data_test[:,2]

In [13]:
y_hat = model.predict(x_test)

In [14]:
incorrect = sum((-1)*(y_test - y_hat.flatten()))
correct = len(y_test) - incorrect
print("Correct: {}".format(correct))
print("Incorrect: {}".format(incorrect))
print("Accuracy: {:2.2%}".format(correct/len(y_test)))

Correct: 382.0
Incorrect: 18.0
Accuracy: 95.50%
